In [ ]:
!pip install -qU \
  langchain-core  \
  langchain[google-genai]  \
  langchain-community  \
  langsmith  \
  google-search-results \
  docarray 

In [ ]:
import getpass
import os
from langchain_google_genai import GoogleGenerativeAI

if not os.environ.get("GOOGLE_API_KEY"):
  os.environ["GOOGLE_API_KEY"] = getpass.getpass("Enter API key for Google Gemini: ")

#llm = GoogleGenerativeAI(model="gemini-2.0-flash", temperature=0.0) 


from langchain.chat_models import init_chat_model

llm = init_chat_model("gemini-2.5-flash", model_provider="google_genai")

In [ ]:
from langchain.prompts import PromptTemplate

prompt_template = "Give me a small report on {topic}"

prompt = PromptTemplate(
    input_variables = ["topic"],
    template = prompt_template
)


In [ ]:
llm_out = llm.invoke("Hello there")

llm_out

In [ ]:
from langchain.schema.output_parser import StrOutputParser

output_parser = StrOutputParser()

In [ ]:
out = output_parser.invoke(llm_out)

out

In [ ]:
from IPython.display import display, Markdown

lcel_chain = ( prompt
              | llm
              | output_parser
              | Markdown
            )

lcel_chain.invoke("retrevial augment generation")

In [ ]:
#TODO: input multiple parameters in replace function

old_word = "RAG"
new_word = "Retrieval Augmented Generation"
def replace(x: str):
    return x.replace(old_word, new_word)
    

lcel_chain = ( prompt
              | llm
              | output_parser
              | replace 
              | Markdown
            )

lcel_chain.invoke("retrevial augment generation")

In [ ]:
from langchain_google_genai import GoogleGenerativeAIEmbeddings 

from langchain.vectorstores import DocArrayInMemorySearch

embedding = GoogleGenerativeAIEmbeddings(model="gemini-embedding-001")

vecstore_a = DocArrayInMemorySearch.from_texts([
    "half the info here",
    "DeepSeek-V3 was released in December 2004"
], embedding=embedding)

vecstore_b = DocArrayInMemorySearch.from_texts([
    "the other half the info here",
    "the DeepSeek-V3 LLM is a mixture of experts model with 671B parameters"
], embedding=embedding)


In [ ]:
prompt_str = """
Using the context provided, answer the user's question.
Context
{context_a}
{context_b}
"""

In [ ]:
from langchain.prompts import ChatPromptTemplate, SystemMessagePromptTemplate, HumanMessagePromptTemplate

prompt = ChatPromptTemplate.from_messages([
    SystemMessagePromptTemplate.from_template(prompt_str),
    HumanMessagePromptTemplate.from_template("{question}")
])

In [ ]:
from langchain_core.runnables import RunnablePassthrough, RunnableParallel

retriever_a = vecstore_a.as_retriever()
retriever_b = vecstore_b.as_retriever()

retreival = RunnableParallel(
    {
        "context_a": retriever_a,
        "context_b": retriever_b,
        "question": RunnablePassthrough()
    }
)


In [ ]:
chain = (
    retreival
    | prompt
    | llm
    | output_parser
)

In [ ]:
chain.invoke("what architecture doest the model DeepSeek released in december use? and how many parameters")